## Setup Python Path

This cell adds the `scripts` directory to the Python path so we can import our custom Telegram loader module. This allows us to use the `TelegramMessageLoader` class we created.

In [1]:
import sys
import os
from pathlib import Path

# Add the scripts directory to the path so we can import our loader
sys.path.append(str(Path.cwd() / "scripts"))

## Environment Configuration

This cell loads the database credentials and configuration from your `.env` file. Make sure your `.env` file contains the correct PostgreSQL connection details for the `telegram_medical` database.

In [2]:
# If your .env is in the project root, this will load it
from dotenv import load_dotenv

load_dotenv(dotenv_path=Path.cwd() / ".env")

False

## Import Custom Loader

This cell dynamically imports our `TelegramMessageLoader` class from the `load_telegram_messages.py` script. This allows us to use the loader functionality directly in the notebook.

In [3]:
import importlib.util
from pathlib import Path

# If running from notebooks/ directory, go up one level to find scripts/
loader_path = Path.cwd().parent / "scripts" / "load_telegram_messages.py"
spec = importlib.util.spec_from_file_location("telegram_loader", loader_path)
telegram_loader = importlib.util.module_from_spec(spec)
spec.loader.exec_module(telegram_loader)

## Initialize Database Loader

This cell creates an instance of the `TelegramMessageLoader` class. The loader will:
- Connect to your PostgreSQL database
- Create the `raw.telegram_messages` table if it doesn't exist
- Set up the database schema and constraints

In [4]:
# Create an instance of the loader
loader = telegram_loader.TelegramMessageLoader()

2025-07-15 02:45:48,465 - INFO - Successfully connected to database: telegram_medical on 127.0.0.1:5432
2025-07-15 02:45:48,490 - INFO - Table raw.telegram_messages created/verified successfully


## Load All Telegram Messages

This cell processes all JSON files in the `notebooks/data/raw/telegram_messages/` directory and loads them into the database. The results show:
- How many messages were inserted from each file
- How many were skipped (duplicates)
- Total processing statistics

In [5]:
# This will process all JSON files in the default directory
results = loader.process_files()

# Show a summary
from pprint import pprint
pprint(results)

2025-07-15 02:45:48,506 - INFO - Found 3 JSON files to process
2025-07-15 02:45:48,506 - INFO - Processing file: c:\Users\Admin\OneDrive\ACADEMIA\10 Academy\Week 7\GitHub Repository\telegram-medical-data-pipeline\notebooks\data\raw\telegram_messages\2025-07-14\CheMed123\CheMed123.json
2025-07-15 02:45:48,579 - INFO - File CheMed123.json: 63 messages inserted, 0 skipped
2025-07-15 02:45:48,587 - INFO - Processing file: c:\Users\Admin\OneDrive\ACADEMIA\10 Academy\Week 7\GitHub Repository\telegram-medical-data-pipeline\notebooks\data\raw\telegram_messages\2025-07-14\lobelia4cosmetics\lobelia4cosmetics.json
2025-07-15 02:45:49,304 - INFO - File lobelia4cosmetics.json: 965 messages inserted, 0 skipped
2025-07-15 02:45:49,304 - INFO - Processing file: c:\Users\Admin\OneDrive\ACADEMIA\10 Academy\Week 7\GitHub Repository\telegram-medical-data-pipeline\notebooks\data\raw\telegram_messages\2025-07-14\tikvahpharma\tikvahpharma.json
2025-07-15 02:45:50,329 - INFO - File tikvahpharma.json: 946 mess

{'c:\\Users\\Admin\\OneDrive\\ACADEMIA\\10 Academy\\Week 7\\GitHub Repository\\telegram-medical-data-pipeline\\notebooks\\data\\raw\\telegram_messages\\2025-07-14\\CheMed123\\CheMed123.json': 63,
 'c:\\Users\\Admin\\OneDrive\\ACADEMIA\\10 Academy\\Week 7\\GitHub Repository\\telegram-medical-data-pipeline\\notebooks\\data\\raw\\telegram_messages\\2025-07-14\\lobelia4cosmetics\\lobelia4cosmetics.json': 965,
 'c:\\Users\\Admin\\OneDrive\\ACADEMIA\\10 Academy\\Week 7\\GitHub Repository\\telegram-medical-data-pipeline\\notebooks\\data\\raw\\telegram_messages\\2025-07-14\\tikvahpharma\\tikvahpharma.json': 946}


## 📋 Summary and Troubleshooting

### What Just Happened?
- ✅ The loader created the `raw.telegram_messages` table in your PostgreSQL database
- ✅ All JSON files were processed and messages were inserted
- ✅ Duplicate messages were automatically skipped using `message_id` as the unique key
- ✅ A detailed log was created in `telegram_loader.log`

### Database Schema Created:
- **Table**: `raw.telegram_messages`
- **Key Fields**: `message_id` (unique), `message_text`, `channel_name`, `message_date`
- **Additional Fields**: `has_media`, `media_type`, `sender_username`, `raw_data` (JSONB)

### Troubleshooting Common Issues:

🔴 **Database Connection Error**
- Check that PostgreSQL is running
- Verify your `.env` file has correct credentials
- Ensure the `telegram_medical` database exists

🔴 **Permission Errors**
- Make sure your PostgreSQL user has rights to create tables and insert data
- Check that the user can access the `telegram_medical` database

�� **Import Errors**
- Run the dependency installation cell
- Check that the `scripts` directory exists and contains the loader file

🔴 **File Not Found Errors**
- Verify JSON files exist in `notebooks/data/raw/telegram_messages/`
- Check file permissions and paths

### Next Steps:
- Explore the loaded data using SQL queries
- Transform and clean the data for analysis
- Create visualizations of the message patterns
- Set up automated data pipelines

## Verify Data Loading

This cell helps you verify that the data was loaded correctly by checking the database and showing some sample records.

In [8]:
# Connect to database and check the loaded data
import pandas as pd
from sqlalchemy import create_engine, text

# Get database connection
db_host = os.getenv('POSTGRES_HOST', 'localhost')
db_port = os.getenv('POSTGRES_PORT', '5432')
db_name = os.getenv('POSTGRES_DB', 'telegram_medical')
db_user = os.getenv('POSTGRES_USER', 'postgres')
db_password = os.getenv('POSTGRES_PASSWORD', '')

database_url = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
engine = create_engine(database_url)

# Check total count
with engine.connect() as conn:
    result = conn.execute(text("SELECT COUNT(*) FROM raw.telegram_messages"))
    total_count = result.fetchone()[0]
    print(f"Total messages loaded: {total_count}")

# Show sample data
df = pd.read_sql("SELECT message_id, channel_name, message_text, message_date FROM raw.telegram_messages LIMIT 5", engine)
print("\nSample messages:")
display(df)

Total messages loaded: 1974

Sample messages:


,message_id,channel_name,message_text,message_date
0,97,CheMed123,"⚠️**Notice!\n**Dear esteemed customers,\nDue t...",2023-02-10 12:23:06
1,96,CheMed123,Mela-One በውስጡ ሆርሞን ያለው ድንገተኛ ወሊድ መቆጣጠርያ ሲሆን ያለ...,2023-02-02 08:58:52
2,95,CheMed123,**አዚትሮማይሲን** በሃኪም መድሃኒት ማዘዣ ከሚታዘዙ አንቲባዮቲኮች አንዱ...,2023-02-01 08:59:37
3,94,CheMed123,**Che-Med Trivia #3\n\n**ምግብና መጠጦች አንዳንድ መድሃኒቶ...,2023-01-31 09:19:53
4,88,CheMed123,🌞**Sundown Cinnamon!\nSupports sugar metabolis...,2023-01-17 08:43:12


## Data Exploration

This cell provides some basic analytics on your loaded Telegram messages to help you understand the data structure and patterns.

In [9]:
# Basic data exploration
import pandas as pd

# Channel distribution
channel_stats = pd.read_sql("""
    SELECT 
        channel_name,
        COUNT(*) as message_count,
        MIN(message_date) as first_message,
        MAX(message_date) as last_message
    FROM raw.telegram_messages 
    GROUP BY channel_name 
    ORDER BY message_count DESC
""", engine)

print("�� Channel Statistics:")
display(channel_stats)

# Messages with media
media_stats = pd.read_sql("""
    SELECT 
        has_media,
        COUNT(*) as count,
        ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM raw.telegram_messages), 2) as percentage
    FROM raw.telegram_messages 
    GROUP BY has_media
""", engine)

print("\n📷 Media Statistics:")
display(media_stats)

# Date range
date_range = pd.read_sql("""
    SELECT 
        MIN(message_date) as earliest_message,
        MAX(message_date) as latest_message,
        COUNT(DISTINCT DATE(message_date)) as unique_days
    FROM raw.telegram_messages
""", engine)

print("\n📅 Date Range:")
display(date_range)

�� Channel Statistics:


,channel_name,message_count,first_message,last_message
0,lobelia4cosmetics,965,2025-05-21 08:33:48,2025-07-14 12:45:26
1,tikvahpharma,946,2025-06-05 08:14:47,2025-07-14 18:27:36
2,CheMed123,63,2022-09-05 09:57:09,2023-02-10 12:23:06



📷 Media Statistics:


,has_media,count,percentage
0,False,581,29.43
1,True,1393,70.57



📅 Date Range:


,earliest_message,latest_message,unique_days
0,2022-09-05 09:57:09,2025-07-14 18:27:36,103
